In [ ]:
from pathlib import Path
import torch

Let's revisit the flowers dataset from the first lesson

In [ ]:
from mads_datasets import DatasetFactoryProvider, DatasetType
from mltrainer.preprocessors import BasePreprocessor

preprocessor = BasePreprocessor()

flowersfactory = DatasetFactoryProvider.create_factory(DatasetType.FLOWERS)
streamers = flowersfactory.create_datastreamer(batchsize=32, preprocessor=preprocessor)

In [ ]:
data = flowersfactory.create_dataset()
train = data["train"]

In [ ]:
train = streamers["train"]
batch = train.batchloop()

In [ ]:
X, y = zip(*batch)

In [ ]:
X[0].shape

In [ ]:
import torch
torch.concatenate(X).shape

In [ ]:
x, y = train[0]

In [ ]:
x.shape, y.shape

In [ ]:
streamers

In [ ]:
train = streamers["train"]
valid = streamers["valid"]
trainstreamer = train.stream()
validstreamer = valid.stream()

The FlowersDatasetFactory adds a transform function that will (at random) flip, scale, affine an image. For the model, this is a new image, so it needs to focus on the general patterns instead of pixels.

In [ ]:
batch = train.batchloop()
len(batch)

In [ ]:
X, y = next(trainstreamer)
X.shape, y.shape


In [ ]:
import matplotlib.pyplot as plt
img = X.permute(0, 2, 3, 1)
fig, axs = plt.subplots(3, 3, figsize=(10,10))
axs = axs.ravel()
for i in range(9):
    axs[i].imshow(img[i])

Let's check the ranges, mean and std of a batch

In [ ]:
X.max(), X.min(), X.mean(), X.std()


Instead of building our own resnet, we will just download a pretrained version. This saves us many hours of training.

In [ ]:
import torchvision
from torchvision.models import resnet18, ResNet18_Weights
resnet = torchvision.models.resnet18(weights=ResNet18_Weights.DEFAULT)


In [ ]:
ResNet18_Weights.DEFAULT

In [ ]:
yhat = resnet(X)
yhat.shape


However, the resnet is trained for 1000 classes. We have just 5...

We will swap the last layer and retrain the model.

First, we freeze all pretrained layers:

In [ ]:
for name, param in resnet.named_parameters():
    param.requires_grad = False


If you study the resnet implementation on [github](https://github.com/pytorch/vision/blob/main/torchvision/models/resnet.py#L206) you can see that the last layer is named `.fc`, like this:

```
 self.fc = nn.Linear(512 * block.expansion, num_classes)
 ```

This is a Linear layer, mapping from 512 * block.expansion to num_classes.


so we will swap that for our own. To do so we need to figure out how many features go into the .fc layer.
We can retrieve the incoming amount of features for the current `.fc` with `.in_features`

In [ ]:
print(type(resnet.fc))
in_features = resnet.fc.in_features
in_features

Let's swap that layer with a small, two layer, neural network

In [ ]:
import torch.nn as nn

resnet.fc = nn.Sequential(
    nn.Linear(in_features, 5)
    # nn.Linear(in_features, 128), nn.ReLU(), nn.Dropout(0.1), nn.Linear(128, 5)
)


In [ ]:
yhat = resnet(X)
yhat.shape


So, we have a fully trained resnet, but we added two layers at the end that transforms everything into 5 classes.
These layers are random, so we need to train them for some epochs

In [ ]:
from src.models import metrics
accuracy = metrics.Accuracy()

This will take some time to train (about 4 min per epoch), you could scale down to amount of trainsteps to speed things up.

You will start with a fairly high learning rate (0.01), and if the learning stops, after patience epochs the learning rate gets halved.

In [ ]:
len(train), len(valid)

In [ ]:
from src.settings import TrainerSettings, ReportTypes

settings = TrainerSettings(
    epochs=10,
    metrics=[accuracy],
    logdir="modellog",
    train_steps=len(train),
    valid_steps=len(valid),
    reporttypes=[ReportTypes.TENSORBOARD],
)

trainer = train_model.Trainer(
    model=resnet,
    settings=settings,
    loss_fn=nn.CrossEntropyLoss(),
    optimizer=torch.optim.Adam,
    traindataloader=trainstreamer,
    validdataloader=validstreamer,
    scheduler=optim.lr_scheduler.ReduceLROnPlateau
    )